In [ ]:
%%capture
import h2o
import h2ohyperopt
h2o.init()

### Data Processing
The test dataset used for demonstrating the capabilities of H2OHyperopt is the titanic dataset. The function ```data()``` is used to preprocess the dataset.

In [ ]:
def data():
    """
    Function to process the example titanic dataset.
 
 Train-Valid-Test split is 60%, 20% and 20% respectively.
    Output
    ---------------------
    trainFr: Training H2OFrame.
    testFr: Test H2OFrame.
    validFr: Validation H2OFrame.
    predictors: List of predictor columns for the Training frame.
    response: String defining the response column for Training frame.
    """
    titanic_df = h2o.import_file(path="https://s3.amazonaws.com/h2o-public-test-data/smalldata/gbm_test/titanic.csv")
    columns_to_be_used = ['pclass', 'age', 'sex', 'sibsp', 'parch', 'ticket',
                          'embarked', 'fare', 'survived']
    columns_to_factorize = ['pclass', 'sex', 'sibsp', 'embarked', 'survived']
    # Factorizing the columns in the columns_to_factorize list
    for col in columns_to_factorize:
        titanic_df[col] = titanic_df[col].asfactor()
    # Selecting only the columns we need
    titanic_frame = titanic_df[columns_to_be_used]
    trainFr, testFr, validFr = titanic_frame.split_frame([0.6, 0.2],
                                                         seed=1234)
    predictors = trainFr.names[:]
    # Removing the response column from the list of predictors
    predictors.remove('survived')
    response = 'survived'
    return trainFr, testFr, validFr, predictors, response

In [ ]:
trainFr, testFr, validFr, predictors, response = data()

### Mutiple Model Type Based Optimization
Let us demonstrate the ModelDocker. Since the problem is a binary classification problem, we specify the metric to AUC. Docking three types of models - GBM's, GLM's and DLE's,

In [ ]:
model_gbm = h2ohyperopt.GBMOptimizer(metric='auc')
# To use the default search space
# model_gbm.select_optimization_parameters("Default")
# To use a combination of Default parameters and the customized parameters.
model_gbm.select_optimization_parameters({'col_sample_rate': 'Default',
                                          'ntrees': 200,
                                          'learn_rate': ('uniform',(0.05, 0.2)),
                                          'nfolds': 7})

In [ ]:
model_dle = h2ohyperopt.DLEOptimizer(metric='auc')
# Selecting parameters to optimize on
model_dle.select_optimization_parameters({'epsilon': 'Default',
                                          'adaptive_rate': True,                                           
                                          'hidden': ('choice', [[10, 20], [30, 40]]),
                                          'nfolds':7})

In [ ]:
model_glm = h2ohyperopt.GLMOptimizer(metric='auc', problemType='Classification')
# Selecting default parameters to optimize on
model_glm.select_optimization_parameters('Default')

### Starting the Optimization
The optimization is started using the function ```start_optimization```. It is necessary to provide a validation frame so that the optimization algorithm can evaluate the model.

In [ ]:
docker = h2ohyperopt.ModelDocker([model_dle, model_gbm, model_glm], 'auc')                                     
docker.start_optimization(num_evals=20, trainingFr=trainFr,
                          validationFr=validFr, response=response,                                              
                          predictors=predictors)

### Metrics available to User

In [ ]:
#To access the best model
bestmodel = docker.best_model

In [ ]:
# To get the best model parameters
docker.best_model_parameters()

In [ ]:
# To get the training and validation scores
docker.best_model_scores()

### Ensembling
Three ensemble methods provided for the user.<br \>
1. Use the best n models from evaluated models
2. Use the best n models from each model class
3. Smart ensembling using correlation

In [ ]:
%%capture
#To use smart ensembling
docker.smart_ensembling()

In [ ]:
testDockerScore = docker.score_ensemble(testFr)

In [ ]:
testDockerScore